# Preprocessing and Quality Control of LC-MS Data with the nPYc-Toolbox

This tutorial demonstrates how to use the LC-MS data processing modules of the nPYc-Toolbox, to import and perform some basic preprocessing and quality control of LC-MS data, and to output a final high quality dataset ready for data modelling. It is based on the quality control criteria previously described in [Lewis et al. 2016](https://www.ncbi.nlm.nih.gov/pubmed/27479709).

Details of how to install all of the required dependencies and to set up your computing environment can be found here [Installing the nPYc-Toolbox](https://npyc-toolbox.readthedocs.io/en/latest/tutorial.html#Installing-the-nPYc-Toolbox).

The dataset used in this example (DEVSET U RPOS xcms) is comprised of six samples of pooled human urine, aliquoted, and independently prepared and measured by ultra-performance liquid chromatography coupled to reversed-phase positive ionisation mode spectrometry (LC-MS, RPOS). Each source sample was separately prepared and assayed thirteen times. A pooled QC sample (study reference, SR) and independent external reference (long-term reference, LTR) of a comparable matrix was also acquired to assist in assessing analytical precision. See [Tutorial Datasets](https://npyc-toolbox.readthedocs.io/en/latest/tutorial.html#tutorial-datasets) and the Metabolights Study [MTBLS694](https://www.ebi.ac.uk/metabolights/MTBLS694) for details of the study, and [Recommended Study Design Elements](https://npyc-toolbox.readthedocs.io/en/latest/studydesign.html) for details of the various QC samples acquired.

The [nPYc-toolbox-tutorials](https://github.com/phenomecentre/nPYc-toolbox-tutorials) contain all of the data required to run the tutorial Jupyter notebooks, full details of which are given [here](https://npyc-toolbox.readthedocs.io/en/latest/tutorial.html#preprocessing-and-quality-control-of-lc-ms-data-with-the-npyc-toolbox).

# 1. Import the nPYc-Toolbox and Configure the Jupyter Notebook

In [ ]:
# Import the nPYc-Toolbox
import nPYc

# Import enumerations for sample type
from nPYc.enumerations import VariableType, DatasetLevel, AssayRole, SampleType

# Import normalisation objects for data normalisation
from nPYc.utilities.normalisation import NullNormaliser, TotalAreaNormaliser, ProbabilisticQuotientNormaliser

# Import matplotlib plotting, configure the Jupyter notebook to plot inline
import matplotlib.pyplot as plt
%matplotlib inline

# Set up plotly to work in offline mode with the notebook
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

# Set up to hide warnings (particularly Depreciation, RunTime warnings, these are not for the user to worry about!)
# These lines can be commented to show warnings if of interest to advanced users
import warnings
warnings.filterwarnings(action='ignore')

# 2. Import LC-MS Data

The next step is to import the LC-MS data into an nPYc-Toolbox [Dataset](https://npyc-toolbox.readthedocs.io/en/latest/objects.html) object.

The nPYc-Toolbox works on feature extracted LC-MS data, where the raw analytical data has been peak-picked into a one dimensional list of detected features, each of which is characterised by abundance and observed m/z and retention time. There are a number of peak-detection algorithms ([Spicer et al. 2017](https://www.ncbi.nlm.nih.gov/pubmed/28890673)), many of which are supported by the nPYc-Toolbox, however in this example, untargeted feature extraction was carried out using [XCMS](https://bioconductor.org/packages/release/bioc/html/xcms.html) [(Smith et al. 2006)](https://www.ncbi.nlm.nih.gov/pubmed/16448051).

In brief, the raw data files were first converted from .RAW format to .mzML format using the msConvert tool from the ProteoWizard tookit ([Chambers et al. 2012](https://www.ncbi.nlm.nih.gov/pubmed/23051804)), for feature-detection with XCMS. msconvert was configured to discard scans below an intensity threshold of 100 (--filter 'threshold absolute 100 most-intense'), and only retain scans from the first acquisition function (--filter 'scanEvent 1'). Subsequent peak detection by XCMS used the centwave algorithm configured with a noise threshold of 600, mass accuracy of 25ppm, and minimum and maximum peakwidth of 1.5 and 5 seconds. Feature grouping was performed using the 'density' method.

The LC-MS XCMS DevSet Dataset is located in 'DEVSET U RPOS xcms.csv', preceding the file path (in quotes) with the letter r ensures that the path is interpreted exactly as written.

The following line creates an object representing the dataset.

In [ ]:
msData = nPYc.MSDataset(r'MTBLS946_features.csv', fileType='XCMS', sop='GenericMS', noFeatureParams=8)

The “sop” parameter points to a configuration file (encoded in JSON format) which contains a set of parameters to use during data import and pre-processing, see [Configuration Files](https://npyc-toolbox.readthedocs.io/en/latest/configuration/configuration.html) for full details.

The default configuration file for LC-MS data, 'GenericMS', contains the recommended parameters for import and quality control of both human urine and plasma/serum data. For a list of all the parameters for MS data, see Table 5 in [Built-in Configuration SOPs](https://npyc-toolbox.readthedocs.io/en/latest/configuration/builtinSOPs.html).

If required, users can create new configuration files, or indeed amend the existing documents with their own values, however, any of the parameters present in these files can also be overwritten by passing values into the data import command directly, without having to modify or generate the configuration files themselves.

For LC-MS data, the majority of the parameters relate to preprocessing at the feature filtering stage, and therefore discussed in more detail below, but as an example, to change the threshold for filtering features based on residual standard deviation (RSD), the argument "rsdThreshold" can be overridden in the following manner:

```
msData = nPYc.MSDataset('DEVSET U RPOS xcms.csv', fileType='XCMS', sop='GenericMS', rsdThreshold=20)
```

Each nPYc-Toolbox Dataset object contains a name that can be changed as shown in the next cell. This name will be used in the summary and visualisation reports and in the file names of the exported data.

In [ ]:
msData.name = 'MTBLS946 dataset'

# 3. Import Sample Metadata and Match to Acquired Data

The default way to add sample metadata is to prepare a CSV file which follows the set of conventions as described in [CSV Template for Metadata Import](https://npyc-toolbox.readthedocs.io/en/latest/samplemetadata.html#csv-template-for-metadata-import) and match it with the acquired data using the 'addSampleInfo' method.

Based on the corresponding entries in the sample metadata CSV file, the acquired samples are categorised into different types, where 'Study Samples (SS)' comprise the main core of the study, and the others are acquired for specific roles in characterising data quality. The main QC samples here are the 'Study Reference (SR)' samples, which comprise a pool of study samples and are used to assess analytical stability across the run, and the 'Long Term Reference (LTR)' samples, a QC sample external to the study. Additionally, the 'Serial Dilution (SRD)' samples are key for assessing linearity of response i.e. that features are measured accurately with respect to their true abundance. For interest we have also included a blank, for full details see [Recommended Study Design Elements](https://npyc-toolbox.readthedocs.io/en/latest/studydesign.html).

Although optional, this is recommended in order to make optimal use of the quality control features and visualisations provided by the nPYc-Toolbox.

An example CSV file is provided, as given in 'DEVSET U RPOS Basic CSV.csv':

In [ ]:
msData.addSampleInfo(descriptionFormat='Basic CSV', filePath=r'MTBLS946_metainfo.csv')

As described in [Datasets](https://npyc-toolbox.readthedocs.io/en/latest/objects.html), the spectral data, sample metadata and feature metadata can be inspected directly using:

```
dataset.intensityData
dataset.sampleMetadata
dataset.featureMetadata

```

In addition to the 'Basic CSV' format, it is also possible to extract analytical parameters directly from the raw data files. In this example, we have restricted data input to the sample metadata CSV file owing to the large file sizes of the raw LC-MS data files. However, if you would like to use this method for the tutorial dataset, the raw files can be downloaded directly from [MTBLS694](https://www.ebi.ac.uk/metabolights/MTBLS694) and the following used to match and extract the analytical parameters:

```
msData.addSampleInfo(descriptionFormat='Raw Data', filePath=r'/path to raw data files')
```

See [Analytical Parameter Extraction](https://npyc-toolbox.readthedocs.io/en/latest/samplemetadata.html#analytical-parameter-extraction) for more details.

In [ ]:
msData.sampleMetadata

In [ ]:
msData.exportDataset(saveFormat='UnifiedCSV', destinationPath='.')